In [1]:
%%capture

%cd ..

%load_ext autoreload
%autoreload 2

from IPython.display import display

In [2]:
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

In [3]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)

In [4]:
import pickle

with open('/home/aris/projects/GEC_EHR/data/interim/proc_sunjae_data.pkl', 'rb') as f:
    df_data = pickle.load(f)

show_df(df_data)

,text
./data/died_SOAP_dataset/001216365_20140930194420.txt,"abdominal discomfort, cough ?viral illness ver..."
./data/died_SOAP_dataset/001078162_20160924192147.txt,"Upper airway pain with breathing, ? due to inf..."
./data/died_SOAP_dataset/000918988_20151104184440.txt,PATIENT is status post sleeve gastrectomy.\nIt...
./data/died_SOAP_dataset/520138305_20161023164639.txt,"URI with viral pharyngitis, clear fluid in R m..."
./data/died_SOAP_dataset/001076295_20141222165037.txt,PATIENT is status post gastric bypass surgery....


(18866, 1)


In [26]:
from critic.harness import *

In [47]:
# model, tokenizer = init_model('gpt2')
# model, tokenizer = init_model('bert-base-uncased')
# model, tokenizer = init_model('emilyalsentzer/Bio_ClinicalBERT')
model, tokenizer = init_model('roberta-base')

Loaded roberta-base


In [48]:
example_sentence = 'I like eating apple.'

critic(model, tokenizer, example_sentence)

Bad! Your sentence log(p) = -145.752
Neighbor sentence with highest log(p): I like having apple. (= -133.932)


(False, -145.751953125, ['I like having apple.', -133.931640625])

In [49]:
example_sentence = 'I like taking apple.'

critic(model, tokenizer, example_sentence)

Bad! Your sentence log(p) = -144.867
Neighbor sentence with highest log(p): I like making apple. (= -132.250)


(False, -144.8671875, ['I like making apple.', -132.25])

In [50]:
_c, _i, _a = 0, 0, 0

for row in df_data.iterrows():
    # print(i[0])
    # print(i[1])
    _path, _text = row
    _text = _text['text']

    sentences = _text.split('\n')

    # print(sentences)
    for sent in sentences:

        is_good, score, counter_example = critic(model, tokenizer, sent, verbose=False)

        if not is_good:
            print('-'*50)
            print(f'{_i}. Sentence:        {sent}')
            print(f'{_i}. Counter example: {counter_example[0]}')
            _i += 1
        _a += 1

    if _c > 5:
        break
    _c += 1

print('*'*50)
print(f'Tested {_a} sentences.')
print(f'{_i} corrections made')

--------------------------------------------------
0. Sentence:        abdominal discomfort, cough ?viral illness versus mild constipation ?reflux related
0. Counter example: abdominal discomfort, cough is? viral illness versus mild constipation? reflux related
--------------------------------------------------
1. Sentence:        Upper airway pain with breathing, ? due to inflammation vs. muscle tightness from running, +/- element of bronchospasm
1. Counter example: Upper arway pain with breathing,? due to inflammation vs. muscle tightness from running, + /- element of bronchospasm
--------------------------------------------------
2. Sentence:        PATIENT is status post sleeve gastrectomy.
2. Counter example: PATIENT his status post sleeve gastrectomy.
--------------------------------------------------
3. Sentence:        It has been 6 months since her surgery.
3. Counter example: Its has been 6 months since her surgery.
--------------------------------------------------
4. Senten

In [43]:
good, bad = [], []
errors = []

for row in df_data.iterrows():
    _path, _text = row
    _text = _text['text']

    sentences = _text.split('\n')

    # print(sentences)
    for sent in sentences:
        try:
            is_good, score, counter_example = gpt2_critic(sent, verbose=False)

            if not is_good:
                bad.append((_path, sent, counter_example[0]))
            else:
                good.append((_path, sent))
        except Exception:
            errors.append((_path, sent))

pp, pn = len(good), len(bad)

print(f'Tested {pp + pn} sentences.')
print(f'{pn} corrections made')
print(f'correction rate: {pn / (pp + pn):.2}')
print(f'{len(errors)} errors encountered.')

Tested 105421 sentences.
38421 corrections made
correction rate: 0.36
231 errors encountered.


In [47]:
# Bug

good[42]

('./data/died_SOAP_dataset/000739056_20140722002949.txt',
 'PATIENT is status post sleeve gastrectomy and it has been 3 weeks since her surgery.\nPATIENT has been mostly compliant with her diet thus far.\nShe is very nearly meeting her protein needs with appropriate items, although she did add in some high sugar syrup in order to make her protein drinks more palatable.\nShe does understand that this is not recommended due to the high calorie and sugar content.\nOtherwise, she is doing well and she has no complaints at this time.\nShe seems willing to listen to all of the education that was provided today and I have no concerns about her moving forward.\n\nPATIENT was provided with stage IV and stage V diet education material.\nShe understands that for the next month, she will be on a stage IV bariatric diet, which consists of all high protein foods.\nShe should not be having any fruits, vegetables or foods high in carbohydrates at this point.\nI would like her to have 3 meals and 3 sna

In [46]:
bad[0]

('./data/died_SOAP_dataset/001078162_20160924192147.txt',
 'Upper airway pain with breathing, ? due to inflammation vs. muscle tightness from running, +/- element of bronchospasm',
 'Upper airway pain with breathing,? due to inflammation vs. muscle tightness from running, + /- elements of bronchospasm')